In [3]:
import json
import numpy
import math

In [29]:
# isi data-dictionary.json 

#{
# 	"p" : 4,
# 	"b" : 8192,
# 	"pegawai" : {
# 		"tabel" : ["no_ktp", "tgl_lahir", "gender", "pendidikan"],
# 		"r" : 128,
# 		"n" : 10000,
# 		"v" : 12,
# 		"br" : 10000
# 	},
# 	"dirawat" : {
# 		"tabel" : ["tgl_dirawat", "status", "periode", "no_inventaris", "no_ktp"],
# 		"r" : 8,
# 		"n" : 150,
# 		"v" : 6,
# 		"br" : 1000
# 	},
# 	"fasilitas" : {
# 		"tabel" : ["no_inventaris", "nama", "jenis", "tgl_dibeli", "pemakaian"],
# 		"r" : 10,
# 		"n" : 1000,
# 		"v" : 7,
# 		"br" : 1000
# 	}
# }

In [30]:
# read file
def readFile(name):
	with open(name) as json_file:
		return json.load(json_file)

data = readFile('data-dictionary.json')

In [31]:
def fanoutRasio(tab):
    return math.floor(data['b'] / (data[tab]['v'] + data['p']))

def blockingFactor(tab):
    return math.floor(data['b'] / data[tab]['r'])

def jumBlokIndeks(r, t):
    if (r>data[t]['n']):
        return 'record tidak tersedia'
    else :
        fanout = fanoutRasio(t)
        return math.ceil(r / fanout)

def jumBlokNonIndeks(r, t):
    if (r>data[t]['n']):
        return 'record tidak tersedia'
    else :
        bfr = blockingFactor(t)
        return math.ceil(r / bfr)
    
def totalBlokData(tab):
    return math.ceil(data[tab]['n'] / blockingFactor(tab))

def totalBlokIndeks(tab):
    return math.ceil(data[tab]['n'] / fanoutRasio(tab))

In [32]:
def menuSatu():
    print('Menu 1 : BFR dan Fan Out Ratio')
    print('')
    
    print('BFR Pegawai : ', blockingFactor('pegawai'))
    print('Fan Out Rasio Pegawai : ', fanoutRasio('pegawai'))
    
    print('BFR Dirawat : ', blockingFactor('dirawat'))
    print('Fan Out Rasio Dirawat : ', fanoutRasio('dirawat'))
    
    print('BFR Fasilitas : ', blockingFactor('fasilitas'))
    print('Fan Out Rasio Fasilitas : ', fanoutRasio('fasilitas'))
    print('')

def menuDua():
    print('Menu 2 : Jumlah Blok')
    print('')
    
    print('Tabel Data Pegawai :', totalBlokData('pegawai'), ' blok')
    print('Indeks Pegawai :', totalBlokIndeks('pegawai'), ' blok')
    
    print('Tabel Data Pegawai :', totalBlokData('dirawat'), ' blok')
    print('Indeks Pegawai :', totalBlokIndeks('dirawat'), ' blok')
    
    print('Tabel Data Pegawai :', totalBlokData('fasilitas'), ' blok')
    print('Indeks Pegawai :', totalBlokIndeks('fasilitas'), ' blok')
    
def menuTiga():
    print('Menu 3 : Pencarian Rekord')
    print('')
    rekord = int(input("Cari Rekord ke : "))
    tabel = input("Nama Tabel : ")

    blok_i = jumBlokIndeks(rekord, tabel)

    print ('Menggunakan indeks, jumlah blok yang diakses : ', blok_i)
    print ('Tanpa indeks, jumlah blok yang diakses : ', jumBlokNonIndeks(rekord, tabel))

In [27]:
menuSatu()
menuDua()
menuTiga()

Menu 1 : BFR dan Fan Out Ratio

BFR Pegawai :  64
Fan Out Rasio Pegawai :  512
BFR Dirawat :  1024
Fan Out Rasio Dirawat :  819
BFR Fasilitas :  819
Fan Out Rasio Fasilitas :  744

Menu 2 : Jumlah Blok

Tabel Data Pegawai : 157  blok
Indeks Pegawai : 20  blok
Tabel Data Pegawai : 1  blok
Indeks Pegawai : 1  blok
Tabel Data Pegawai : 2  blok
Indeks Pegawai : 2  blok
Menu 3 : Pencarian Rekord

Cari Rekord ke : 1243
Nama Tabel : pegawai
Menggunakan indeks, jumlah blok yang diakses :  3
Tanpa indeks, jumlah blok yang diakses :  20


In [35]:
print('Menu 4 : QEP dan COST')
print('')

#qep where
array_query = {}
array_query['kolom'] = ['no_inventaris','nama']
array_query['tabel'] = ['fasilitas']
array_query['where'] = ['no_inventaris = 1']
def qep(query):
    if (query['kolom'][0] != '*'):
        return { 
            'projection' : query['kolom'],
            'selection' : query['where'],
            'tabel' : query['tabel']
        }
    else:
        return {
            'selection' : query['where'],
            'tabel' : query['tabel']
        }

#cost where, diisi ya geng hehe
def costA1nkey(t):
    return data[t]['br']

def costA1key(t):
    return data[t]['br']/2

def costA2(t):
    return math.ceil(math.log(data[t]['br'],fanoutRasio(t))) + 1

def costBNLJ(inner,outer):
    return data[inner]['br']*data[outer]['br'] + data[outer]['br']

#qep join, diisi ya geng hehe

#cost join, diisi ya geng hehe

#tes qep where
if(array_query['kolom']!='*'):
    print('PROJECTION', qep(array_query)['projection'], ',' , qep(array_query)['projection'])
print('SELECTION', qep(array_query)['selection'])
print(qep(array_query)['tabel'])

h = costA1key(array_query['tabel'])
print(h)
print(costBNLJ('fasilitas','pegawai'))

Menu 4 : QEP dan COST

PROJECTION ['no_inventaris', 'nama'] , ['no_inventaris', 'nama']
SELECTION ['no_inventaris = 1']
['fasilitas']


TypeError: unhashable type: 'list'

In [ ]:
def sharedPool:
    